# Income EDA
Exploring Salary data in the United States by using the BLS API
[Getting Strated with BLS API](URL 'https://www.bls.gov/developers/')


In [20]:
#Importing Dependencies
import json
import requests
import prettytable
import pandas as pd

In [ ]:
# retrieving mapping of survey abbreviations to survey names from BLS
headers = {'content-type': 'application/json'}
#data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],'latest':'latest'}) # retrieving latest data from a series id
p = requests.post('https://api.bls.gov/publicAPI/v2/surveys/')# data=data, headers=headers)
json_data = json.loads(p.text)
df = pd.DataFrame(json_data['Results']['survey'])

# Writing the resulting request to a CSV for manual inspection
df.to_csv('Unique_Surveys.csv')

In [12]:
# From the BLS website we know the database we are interseted in is 'Wage Modeling'
# We search for its abbreviation by passing the survey name to the dataframe
abbreviation = df[df['survey_name'] == 'Wage Modeling'].iloc[0,0]
print(f'The abbreviation for the wage modeling database is {abbreviation}')


The abbreviation for the wage modeling database is WM


To collect data on the wage estimation from the BLS I need to know the series ID of the data I am looking for. Unfortunately, the BLS does not contain a series ID catalogue and I will have to learn the syntax of series ID's withing the Wage Modeling survey to learn how to access all data.

Update: its possible to get a complete dataset of all wage estimates for the different occupations and levels that they handle. The dataset contains the series ID for each estimate they have. We can find the unique ID's and pass them as arguements to the API Call to get the data through the API.

In [8]:
# Read in 2020 wage estimate dataset and extract unique series ID
wageEstimate_DF = pd.read_excel('mwe-2020complete.xlsx', sheet_name="MWE 2020")

In [46]:
# Reviewing the columns present in the dataset
wageEstimate_DF.columns

# Filtering out columns we are not interested in
wageEstimate_DF = wageEstimate_DF[['Series title', 'Area text', 'Occupation text',
       'Average hourly wage\n($)',
       'Series ID', 'Area code', 'Occupation \ncode']]


In [55]:
# Extract the State component from 'Area text' and add as a new column
from posixpath import split


splitColumns = wageEstimate_DF['Area text'].str.split(pat=',',expand=True)
wageEstimate_DF[['Sub-Region', 'State']] = splitColumns

In [60]:
wageEstimate_DF['State'].unique()
# 93 unique state instances, there should be roughly 50
# Some states are aggregates of up to 3 states
# Need to investigate why they are combined, and determine a method of binning them into a single state

array([None, ' TX', ' OH', ' GA', ' OR', ' NY', ' NM', ' LA', ' PA-NJ',
       ' PA', ' IA', ' AK', ' MI', ' AL', ' WI', ' NC', ' NJ', ' GA-SC',
       ' CA', ' MD', ' WV', ' WA', ' MT', ' ND', ' VA', ' IL', ' IN',
       ' ID', ' CO', ' KY', ' FL', ' MO-IL', ' NV', ' WY', ' SC',
       ' NC-SC', ' TN-GA', ' IL-IN-WI', ' OH-KY-IN', ' TN-KY', ' TN',
       ' MO', ' GA-AL', ' MD-WV', ' IA-IL', ' DE', ' MN-WI', ' OK',
       ' IN-KY', ' ND-MN', ' AR-MO', ' AZ', ' AR-OK', ' NE', ' MS', ' AR',
       ' WV-KY-OH', ' HI', ' MO-KS', ' TN-VA', ' WI-MN', ' KS', ' ID-WA',
       ' UT-ID', ' KY-IN', ' MN', ' TN-MS-AR', ' SC-NC', ' NY-NJ-PA',
       ' UT', ' NE-IA', ' PA-NJ-DE-MD', ' OR-WA', ' SD', ' MD-DE',
       ' IA-NE-SD', ' IN-MI', ' TX-AR', ' VA-NC', ' DC-VA-MD-WV',
       ' WV-OH', ' VA-WV', ' OH-PA', ' ME', ' MA', ' MA-NH', ' CT', ' VT',
       ' NH-ME', ' NH', ' CT-RI', ' RI-MA', ' MA-CT'], dtype=object)